# Evaluating LLM-based tasks

Evaluating LLM-based use cases is pivotal for several reasons.
First, with the myriad of methods available, comparability becomes essential.
By systematically evaluating different approaches, we can discern which techniques are more effective or suited for specific tasks, fostering a deeper understanding of their strengths and weaknesses.
Secondly, optimization plays a significant role. Without proper evaluation metrics and rigorous testing, it becomes challenging to fine-tune methods and/or models to achieve their maximum potential.
Moreover, drawing comparisons with state-of-the-art (SOTA) and open-source methods is crucial.
Such comparisons not only provide benchmarks but also enable users to determine the value-added by proprietary or newer models over freely available counterparts.

However, evaluating LLMs, especially in the domain of text generation, presents unique challenges.
Text generation is inherently subjective, and what one evaluator deems coherent and relevant, another might find disjointed or off-topic. This subjectivity complicates the establishment of universal evaluation standards, making it imperative to approach LLM evaluation with a multifaceted and comprehensive strategy.

### Evaluating classification use-cases

To (at least for now) evade the elusive issue described in the last paragraph, let's have a look at an easier to evaluate methodology: classification.
Why is this easier?
Well, unlike other tasks such as QA, the result of a classification task is more or less binary (true/false).
There are very few grey areas, as it is unlikely that a classification result is somewhat or "half" correct.

Make sure that you have familiarized yourself with the `PromptBasedClassify` prior to starting this notebook.


First, we need to instantiate our task, an evaluator for it and a repository that stores the evaluation results along with tracing information for the evaluated examples.


In [2]:
import os

from dotenv import load_dotenv

from intelligence_layer.connectors import LimitedConcurrencyClient
from intelligence_layer.core import InMemoryEvaluationRepository, InMemoryDatasetRepository, Runner
from intelligence_layer.use_cases import SingleLabelClassifyEvaluator, PromptBasedClassify

load_dotenv()

client = LimitedConcurrencyClient.from_token(os.getenv("AA_TOKEN"))
task = PromptBasedClassify(client)
evaluation_repository = InMemoryEvaluationRepository()
dataset_repository = InMemoryDatasetRepository()


evaluator = SingleLabelClassifyEvaluator(evaluation_repository, dataset_repository, "singel-label-classify")
runner = Runner(task, evaluation_repository, dataset_repository, "prompt-based-classify")


Now, let's run a single example and see what comes of it!

In [16]:
from intelligence_layer.core import Chunk, NoOpTracer
from intelligence_layer.use_cases import ClassifyInput
from intelligence_layer.core import Example


classify_input = ClassifyInput(
        chunk=Chunk("This is good"),
        labels=frozenset({"positive", "negative"}),
    )

single_example_dataset = dataset_repository.create_dataset(examples=[
    Example(
        input=classify_input,
        expected_output="positive"
    )
])

run_overview = runner.run_dataset(single_example_dataset, NoOpTracer())
evaluation_overview = evaluator.evaluate_dataset(run_overview.id)

print("Statistics: ", evaluation_overview.statistics)



Evaluating: 1it [00:00,  7.96it/s]

Statistics:  percentage_correct=1.0


Cool!

Let's have a look at this [dataset](https://huggingface.co/cardiffnlp/tweet-topic-21-multi) for more elaborate evaluation.

In [4]:
from datasets import load_dataset

dataset = load_dataset(f"cardiffnlp/tweet_topic_multi")
test_set_name = "validation_random"
all_data = list(dataset[test_set_name])
data = all_data[:25] # this has 573 datapoints, let's take a look at 25 for now


Generating test_2020 split: 0 examples [00:00, ? examples/s]

Generating test_2021 split: 0 examples [00:00, ? examples/s]

Generating train_2020 split: 0 examples [00:00, ? examples/s]

Generating train_2021 split: 0 examples [00:00, ? examples/s]

Generating train_all split: 0 examples [00:00, ? examples/s]

...

Generating validation_random split: 0 examples [00:00, ? examples/s]

Generating test_coling2022_random split: 0 examples [00:00, ? examples/s]

Generating train_coling2022_random split: 0 examples [00:00, ? examples/s]

Generating test_coling2022 split: 0 examples [00:00, ? examples/s]

Generating train_coling2022 split: 0 examples [00:00, ? examples/s]

We need to transform our dataset into the required format. 
Therefore, let's check out what it looks like.

In [5]:
data[1]


{'text': 'COMING UP!! At the beginning of the pandemic, various models predicted a doomsday scenario for Nigeria & other African countries, yet fewer cases than expected have been recorded. For the latest on Nigeria’s response to #COVID19, join DG… {{URL}} VIA {@NCDC@} ',
 'date': '2020-08-17',
 'label': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 'label_name': ['news_&_social_concern'],
 'id': '1295296955732471809'}

Accordingly, this must be translated into the interface of our `Evaluator`.

This is the target structure:

``` python
class Example(BaseModel, Generic[Input, ExpectedOutput]):
    input: Input
    expected_output: ExpectedOutput
    id: Optional[str] = Field(default_factory=lambda: str(uuid4()))

```

We want the `input` in each `Example` to mimic the input of an actual task, therefore we must every time include the text (chunk) and all possible labels.
The `expected_output` shall correspond to anything we wish to compare our generated output to.
In this case, that means the correct class(es).

In [6]:


all_labels = list(set(c for d in data for c in d["label_name"]))
dataset_id = dataset_repository.create_dataset(
    examples=[
        Example(
            input=ClassifyInput(
                chunk=Chunk(d["text"]),
                labels=all_labels
            ),
            expected_output=d["label_name"]
        ) for d in data
    ]
)


Ok, let's run this!

Note that this may take a while as we parallelise the tasks in a way that accommodates the inference API.

In [7]:
run_overview = runner.run_dataset(dataset_id)
evaluation_overview = evaluator.evaluate_dataset(run_overview.id)
evaluation_overview.raise_on_evaluation_failure()

Evaluating: 25it [00:04,  5.52it/s]


Checking out the results...

In [13]:
from intelligence_layer.use_cases import SingleLabelClassifyEvaluation

print("Percentage correct:", evaluation_overview.statistics.percentage_correct)
print("First example:", evaluation_repository.example_evaluations(eval_id=evaluation_overview.id, evaluation_type=SingleLabelClassifyEvaluation)[0])

Percentage correct: 0.6
First example: eval_id='78540c57-3aa0-4c1c-8a9f-89381b76015e' example_id='09cf9810-4b2b-428c-aed9-56f22c75e586' result=SingleLabelClassifyEvaluation(correct=True)


For the sake of comparison, let's see if we can achieve a better result with our EmbeddingBasedClassifier.
Here, we have to provide some example for each class.

We can even reuse our evaluation and dataset repositories.

In [ ]:
from collections import defaultdict
from typing import Any, Mapping, Sequence
from intelligence_layer.use_cases import MultiLabelClassifyEvaluator, EmbeddingBasedClassify, LabelWithExamples

def build_labels_and_examples(hf_data: Any) -> Mapping[str, Sequence[str]]:
    examples = defaultdict(list)
    for d in hf_data:
        labels = d["label_name"]
        for label in labels:
            if len(examples[label]) < 20:
                examples[label].append(d["text"])
    return examples

embedding_based_classify = EmbeddingBasedClassify(
    client,
    labels_with_examples=[
        LabelWithExamples(
            name=name,
            examples=examples
        ) for name, examples in build_labels_and_examples(all_data[25:]).items()
    ]
)

embedding_based_classify_evaluator = MultiLabelClassifyEvaluator(evaluation_repository, dataset_repository, "multi-label-classify", threshold=0.6)
embedding_based_classify_runner = Runner(embedding_based_classify, evaluation_repository, dataset_repository, "embedding-based-classify")


In [ ]:
embedding_based_classify_run_result = embedding_based_classify_runner.run_dataset(dataset_id)
embedding_based_classify_evaluation_result = embedding_based_classify_evaluator.evaluate_dataset(embedding_based_classify_run_result.id)
embedding_based_classify_evaluation_result.raise_on_evaluation_failure()

In [ ]:
embedding_based_classify_evaluation_result.statistics.macro_avg

Apparently, our method has a great recall value, but we tend to falsely predict labels at times.

Note, that the evaluation criteria for the multiple label approach are a lot harsher; we evaluate whether we correctly predict all labels & not just one of the correct ones!



### Wrap up

There you go, this is how to evaluate any task using the 'Intelligence Layer'-framework.
Simply define an `Evaluator` that takes the target `Task` as input and customize the `do_evaluate` as well as `aggregate` methods.